In [1]:
import sys
sys.path.append('../template_model')
from reading_thiagos_templates import load_dataset

# Load data

In [2]:
data = {{'train': 'treinamento',
         'dev': 'desenvolvimento',
         'test': 'teste'
        }[db_name]: load_dataset(db_name) for db_name in ['train', 'dev', 'test']}

# Gather statistics

\todo{Apresentar estatísticas sobre os dados da WebNLG - quantidade de triplas distintas, por subconjunto; de lexes, predicados, sujeitos, objetos; quantidade de textos por predicado; predicados mais frequentes; o mesmo com sujeito/objeto}

## Quantidade de triplas distintas

In [3]:
from more_itertools import flatten
import pandas as pd
from collections import Counter
%matplotlib inline

In [4]:
triplas_distintas = {db_name: len(set(flatten(e.triples for e in db))) for db_name, db in data.items()}

In [5]:
lexes_distintas = {db_name: len(set(flatten([l['text'] for l in e.lexes] for e in db))) for db_name, db in data.items()}

In [6]:
pred_distintos = {db_name: len(set(flatten([t.predicate for t in e.triples] for e in db))) for db_name, db in data.items()}

In [7]:
sub_distintos = {db_name: len(set(flatten([t.subject for t in e.triples] for e in db))) for db_name, db in data.items()}

In [8]:
obj_distintos = {db_name: len(set(flatten([t.object for t in e.triples] for e in db))) for db_name, db in data.items()}

In [9]:
def get_most_freqs(f):
    texts_per_f = dict()

    for db_name, db in data.items():
        n_texts = Counter()
        for e in db:
            for t in e.triples:
                n_texts[f(t)] += len(e.lexes)
        texts_per_f[db_name] = pd.Series(n_texts)
    
    most_freq_f = {db_name: '\n'.join(top.nlargest(3).index.tolist()) for db_name, top in texts_per_f.items()}
    
    return texts_per_f, most_freq_f

In [10]:
texts_per_predicate, most_freq_predicates = get_most_freqs(lambda t: t.predicate)
texts_per_subject, most_freq_subjects = get_most_freqs(lambda t: t.subject)
texts_per_object, most_freq_objects = get_most_freqs(lambda t: t.object)

In [11]:
median_texts_per_subject = {db_name: s.median() for db_name, s in texts_per_subject.items()}
median_texts_per_predicate = {db_name: s.median() for db_name, s in texts_per_predicate.items()}
median_texts_per_object = {db_name: s.median() for db_name, s in texts_per_object.items()}

# Joining everything

In [17]:
df = pd.DataFrame([triplas_distintas,
              lexes_distintas,
              sub_distintos,
              pred_distintos,
              obj_distintos,
              #most_freq_subjects,
              #most_freq_predicates,
              #most_freq_objects,
              median_texts_per_subject,
              median_texts_per_predicate,
              median_texts_per_object], 
             index=['Triplas distintas',
                    'Textos de referência distintos',
                    'Sujeitos distintos',
                    'Predicados distintos',
                    'Objetos distintos',
                    #'Sujeitos mais frequentes',
                    #'Predicados mais frequentes',
                    #'Objetos mais frequentes',
                    'Mediana de textos associados a sujeitos',
                    'Mediana de textos associados a predicados',
                    'Mediana de textos associados a objetos'])

df[['treinamento', 'desenvolvimento', 'teste']]

,treinamento,desenvolvimento,teste
Triplas distintas,2090.0,1102.0,2331.0
Textos de referência distintos,18051.0,2261.0,4922.0
Sujeitos distintos,430.0,304.0,575.0
Predicados distintos,246.0,186.0,300.0
Objetos distintos,1619.0,918.0,1888.0
Mediana de textos associados a sujeitos,51.0,12.0,14.0
Mediana de textos associados a predicados,59.0,13.0,15.0
Mediana de textos associados a objetos,18.0,5.0,5.0


In [28]:
print(df[['treinamento', 'desenvolvimento', 'teste']].to_latex(float_format='%.0f'))

\begin{tabular}{lrrr}
\toprule
{} &  treinamento &  desenvolvimento &  teste \\
\midrule
Triplas distintas                         &         2090 &             1102 &   2331 \\
Textos de referência distintos            &        18051 &             2261 &   4922 \\
Sujeitos distintos                        &          430 &              304 &    575 \\
Predicados distintos                      &          246 &              186 &    300 \\
Objetos distintos                         &         1619 &              918 &   1888 \\
Mediana de textos associados a sujeitos   &           51 &               12 &     14 \\
Mediana de textos associados a predicados &           59 &               13 &     15 \\
Mediana de textos associados a objetos    &           18 &                5 &      5 \\
\bottomrule
\end{tabular}



In [21]:
df.to_latex?